In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv

In [5]:
df1 = pd.read_csv('/Users/hmead/Desktop/BW2/General characteristics of people.csv', sep=';',engine='python', error_bad_lines=False, encoding= 'latin')
df2 = pd.read_csv('/Users/hmead/Desktop/BW2/Datos de la vivienda.csv', sep=';',engine='python', error_bad_lines=False, encoding= 'latin')
df3 = pd.read_csv('/Users/hmead/Desktop/BW2/Perception of security and coexistence.csv', sep=';',engine='python', error_bad_lines=False, encoding= 'latin')

In [115]:
general = df1.copy()
housing = df2.copy()
security = df3.copy()

In [116]:
general_columns = {
            'ï»¿DIRECTORIO':'id', 'SECUENCIA_ENCUESTA':'survey_sequence','SECUENCIA_P':'sequence_p',
            'ORDEN':'order', 'P220' : 'sex', 'P5785': 'age', 'P5501':'relationship_to_head_of_household',
            'P6210':'education_level','P1366': 'relationship_status','P1402':'time_lived_in_city',
            'P1403':'time_lived_in_neighborhood','P1365': 'activity','P1364':'activity_location',
            'P1363': 'activity_time_of_day', 'FEX_C': 'gen_expansion'
              }

housing_columns = {
        'ï»¿DIRECTORIO':'id','SECUENCIA_ENCUESTA':'survey_sequence','SECUENCIA_P':'sequence_p',
        'ORDEN':'order', 'P5747': 'dwelling_type','P5752':'electricity_service', 
        'P5752S1':'social_stratum', 'P5090':'own_rent_usufruct_other','DEPMUNI':'dept_muni',
        'FEX_C': 'housing_expansion'
          }

security_columns = {
          'ï»¿DIRECTORIO':'id','SECUENCIA_ENCUESTA':'survey_sequence','SECUENCIA_P':'sequence_p',
          'P1362':'general_safe_unsafe', 'P1359':'city_safe_unsafe', 'P1594': 'city_security_inc_dec_same',
          'P564':'will_be_victim_next_12_months', 'P562':'how_unsafe', 'P1356S1': 'activity_location_safe_unsafe', 
          'P1356S2':'public_parks_recreational_sports_spaces', 'P1356S3':'market_squares_shopping_streets', 
          'P1356S4': 'public_transportation', 'P1356S5':'ATMs', 'P1356S6':'public_roads', 'P1356S8':'pedestrian_bridges', 
          'P1356S7':'nightclubs_bars_entertainment_venues', 'P1116':'measures_taken', 'P1353S1':'justify_violence_family', 
          'P1353S2':'justify_violence_punishment', 'P1353S3':'justify_violence_self-defense',
          'P1117':'how_certain_will_be_victim_next_12_months', 
          'P1353S4':'justify_violence_infidelity', 'P1353S5':'justify_violence_defend_stranger', 
          'P1353S6':'justify_violence_defend_property', 'P1115': 'action_taken_as_witness', 'FEX_C': 'sec_expansion'
            }

In [117]:
general = general.rename(columns = general_columns)
housing = housing.rename(columns = housing_columns)
security = security.rename(columns = security_columns)

In [118]:
# Housing dataset only has data from:

   # 'sequence_p' ==1        &
   # 'survey_sequence']==1

# Security dataset only has data from:
   # 'age' >14

condition1 = general['age']>14

condition2_g = general['sequence_p'] ==1
condition3_g = general['survey_sequence']==1

condition2_s = security['sequence_p'] ==1
condition3_s = security['survey_sequence']==1

# Apply conditions, in preparation for horizontal merge

general = general[condition1]
general = general[condition2_g]
general = general[condition3_g]

security = security[condition2_s]
security = security[condition3_s]

# Confirm rows

print(general.shape)
print(housing.shape)
print(security.shape);

(64406, 16)
(64406, 10)
(64406, 49)


<ipython-input-118-67b36480d18c>:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  general = general[condition2_g]
<ipython-input-118-67b36480d18c>:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  general = general[condition3_g]
<ipython-input-118-67b36480d18c>:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  security = security[condition3_s]


In [119]:
# Drop Columns with lots of NaNs

general_drop_columns = ['P6210S1', 'gen_expansion', 'survey_sequence', 'sequence_p',
                       'activity_location', 'activity_time_of_day', ]

housing_drop_columns = ['housing_expansion', 'survey_sequence', 'sequence_p']

security_drop_columns = ['how_unsafe','how_certain_will_be_victim_next_12_months',
                         'P1361S1','P1361S2','P1361S3','P1361S4','P1361S5','P1361S6',
                         'P1361S7','P1361S8','P1361S9', 'P563','P1358S1','P1358S10',
                         'P1358S2','P1358S3','P1358S4','P1358S5','P1358S6','P1358S7',
                         'P1361S1', 'P1361S2', 'P1361S3', 'P1361S4', 'survey_sequence',
                         'sequence_p','P1358S8', 'P1358S9', 'P1361S10','P1358S11', 
                          'P1361S10','sec_expansion','ORDEN']

In [120]:
general = general.drop(general_drop_columns, axis=1)

housing = housing.drop(housing_drop_columns, axis=1)

security = security.drop(security_drop_columns, axis=1)

In [121]:
# Merge into one dataframe

df = general.merge(housing, on='id').merge(security, on='id')

df.shape

(64406, 36)

In [122]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df = df.drop(col,axis=1)
df.shape

(64406, 33)

In [124]:
import numpy as np

df = df.replace(' ', np.NaN)

In [125]:
df = df[df.social_stratum.notnull()]

In [126]:
objects = df.select_dtypes('object').columns.tolist()
df[objects] = df[objects].astype('float')

In [127]:
column_list = df.nunique().index.tolist()

In [128]:
target = column_list[13]
leakage = column_list[13:23]

In [129]:
df[target].value_counts(normalize=True)

1    0.713263
2    0.286737
Name: general_safe_unsafe, dtype: float64

In [130]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, train_size=0.85, test_size=0.15, 
                              stratify=df['general_safe_unsafe'], random_state=42)

train, val = train_test_split(train, train_size=0.85, test_size=0.15, 
                              stratify=train['general_safe_unsafe'], random_state=42)

In [131]:
print(train.shape)
print(val.shape)
print(test.shape)

(46325, 33)
(8175, 33)
(9618, 33)


In [132]:
y_train = train[target]
y_val = val[target]
y_test = test[target]

X_train = train.drop(train[leakage], axis=1)
X_val = val.drop(val[leakage], axis=1)
X_test = test.drop(test[leakage], axis=1)

print(y_train.shape)
print(X_train.shape)

print(y_val.shape)
print(X_val.shape)

print(y_test.shape)
print(X_test.shape)

(46325,)
(46325, 23)
(8175,)
(8175, 23)
(9618,)
(9618, 23)


In [138]:
from sklearn.ensemble import RandomForestClassifier

model = (
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

# Fit on train, score on val
model.fit(X_train, y_train)

print('Validation Accuracy', model.score(X_val, y_val))
print('\nTest Accuracy', model.score(X_test, y_test))



Validation Accuracy 0.744954128440367

Test Accuracy 0.7425660220420046
